# Set up and global variables

In [ ]:
from pathlib import Path

import os

from IPython.display import display, HTML

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import src.ipython_loader as loader

In [ ]:
os.environ["CONFIG_ENV"] = "debug"
if False:
    os.environ["CONFIG_ENV"] = "production"

from config import load_config
config = load_config()

DEBUG = config["DEBUG"]
RESOLUTION = config['DEFAULTS']['resolution']

# input data
STUDY_RESULTS_PATH = config['PATHS']['student_study_results']

# output data
IMAGE_DIR = config['PATHS']['images'] / 'student_study_preparation'

os.makedirs(IMAGE_DIR, exist_ok=True)

***

# Loading data

## Parsing utils

In [ ]:
# Parsing utils
def _normalize_model_versions(x):
    if x == "Version A":
        return "model"
    elif x == "Version B":
        return "baseline"
    # neutral or undecided
    return "neutral"

def _normalize_preference_category(x):
    if x == "The ordering made more sense.":
        return "order"
    elif x == "The explanations were clearer.":
        return "explanation"
    elif x == "I understood one version more clearly.":
        return "clarity"
    return "other"

def _normalize_explanations_category(x):
    if x == "Not really.":
        return "no"
    elif x == "Somewhat.":
        return "somewhat"
    elif x == "Yes, definitely.":
        return "yes"
    raise ValueError(f"{x} not recognized.")
    
def _normalize_fix_category(x):
    if x == "Yes, it would help.":
        return "yes"
    elif x == "No, I would ignore it.":
        return "no"
    elif x == "Sometimes.":
        return "somewhat"
    raise ValueError(f"{x} not recognized.")
    
def _normalize_understood(x):
    """Set manually based on inspection of the data."""
    x = str(x).lower()
    if "yes" in x or "hope" in x or "mostly" in x:
        return "yes"
    if "no" in x:
        return "no"
    return "maybe"

## Data

In [ ]:
original_final = pd.read_csv(STUDY_RESULTS_PATH / "final.csv")
original_submissions = pd.read_csv(STUDY_RESULTS_PATH / "per_submission.csv")

per_submission_df = pd.DataFrame()

per_submission_df['time'] = pd.to_datetime(
    original_submissions["Timestamp"],
    format="%Y/%m/%d %I:%M:%S %p %Z",
    errors="coerce"
)

per_submission_df["prefer_model"] = original_submissions["Which version did you prefer?"].apply(_normalize_model_versions)
per_submission_df["helps_first"] = original_submissions["Which version would help you decide what to fix first?"].apply(_normalize_model_versions)
per_submission_df["top_defect"] = original_submissions["Focusing only on the first defect, which version ranked it better?"].apply(_normalize_model_versions)

per_submission_df["confidence"] = pd.to_numeric(original_submissions["How confident you are in your choices?"], errors="coerce")

final_df = pd.DataFrame()

final_df['Timestamp'] = pd.to_datetime(
    original_final["Timestamp"],
    format="%Y/%m/%d %I:%M:%S %p %Z",
    errors="coerce"
)

final_df["Student ID"] = original_final["Student ID"].fillna("Unknown")

final_df["why_version"] = original_final["What made you prefer one version over the other?"].apply(_normalize_preference_category)
final_df["explanations_helped"] = original_final["Did the explanations help you understand why defects were ordered in that way?"].apply(_normalize_explanations_category)
final_df["ordering_effect"] = original_final["Did the ordering affect how you would approach fixing the code?"].apply(_normalize_fix_category)
final_df["confused_tasks"] = original_final["Were any tasks confusing?"].apply(_normalize_understood)
final_df['confused_defects'] = original_final["Did you understand all the defects?"].apply(_normalize_understood)
final_df["comments"] = original_final["Do you have any additional comments?"]

In [ ]:
final_df.head(2)

In [ ]:
per_submission_df.head(2)